In [13]:
import io
import progressbar
import os

In [14]:
class Constants:
    def __init__(self, max_bits = 18, ceil = 7):
        self.max_bits = max_bits
        self.ceil = ceil

In [105]:
class LZW:
    def __init__(self, max_bits = 18, ceil = 7):
        self.constants = Constants(max_bits, ceil)
        self.letters_counts={}
        self.file_str = ''
        self.file_path = ''
        self.codes = {}
        self.out_codes = {}
        self.output = ''
        self.decoded = ''
        self.max_length = 2**self.constants.max_bits
        self.main_length = 0
        self.ceil = self.constants.ceil
        self.main_codes = {}
        self.main_out_codes = {}
        self.encoded_string = ''
    
    
    def readFile(self, file_path, Str = ''):
        self.file_path = file_path
        if Str == '': 
            self.file_str = io.open(file_path, mode="r", encoding="utf-8").read()
            self.file_str = self.file_str.replace('\n','\r\n')
        else:
            self.file_str = Str
        let_count = 1
        for let in self.file_str:
            if self.letters_counts.has_key(let):
                self.letters_counts[let] = self.letters_counts[let] + 1
            else:
                self.letters_counts[let] = 1
                self.codes[let] = len(self.codes)
        self.out_codes = {v: k for k, v in self.codes.iteritems()}
        self.main_codes = self.codes.copy()
        self.main_out_codes = self.out_codes.copy()
        self.main_length = len(self.codes)
                
            
    def encode(self):
        s = self.file_str[0]
        i = 1
        with progressbar.ProgressBar(max_value=len(self.file_str)) as bar:
            while i<len(self.file_str):
                c = self.file_str[i]
                if self.codes.get(s+c, None) != None:
                    s = s+c
                else:
                    bits = bin(self.codes[s])[2:]
                    self.output += bits.zfill(self.ceil)
                    self.codes[s+c] = len(self.codes)
                    s = c
                    
                if len(self.codes) > 2**self.ceil:
                    self.ceil += 1

                if len(self.codes) >= self.max_length:
                    self.codes = self.main_codes.copy()
                    self.ceil = self.constants.ceil

                i += 1
                bar.update(i)
        bits = bin(self.codes[s])[2:]
        self.output += bits.zfill(self.ceil)
        
        
    def decode(self):
        self.ceil = self.constants.ceil
        s = None
        i = 0
        with progressbar.ProgressBar(max_value=len(self.output)) as bar:
            while i< len(self.output):
                if i + self.ceil > len(self.output):
                    break
                k = int(self.output[i: i+self.ceil], 2)
                try:
                    entry = self.out_codes[k]
                except KeyError:
                    entry = s + s[0]

                self.decoded += entry

                if s != None:
                    self.out_codes[len(self.out_codes)] = s+entry[0]
                    
                if len(self.out_codes) >= 2**self.ceil:
                    self.ceil += 1
                    i -= 1

                if len(self.out_codes) >= self.max_length:
                    self.out_codes = self.main_out_codes.copy()
                    self.ceil = self.constants.ceil

                s = entry
                if i >= 0:
                    bar.update(i)
                i += self.ceil
                
    
    def printEncodedFile(self, file_name = 'encoded_file.bin'):
        byte_array = []
        i = 0
        while i < len(self.output):
            byte_array.append(int(self.output[i: i+8], 2))
            i += 8
        rest_bits = len(self.output) % 8
        if rest_bits != 0:
            byte_array[len(byte_array) - 1] = byte_array[len(byte_array) - 1] << (8 - rest_bits)
        byte_array = bytearray(byte_array)
        newFile = open(file_name, 'wb')
        newFile.write(byte_array)
        newFile.close()
        
        
    def readEncodedFile(self, file_name = 'encoded_file.bin'):
        f = open(file_name, 'rb')
        byte_array = f.read()
        self.encoded_string = ''
        i = 0
        with progressbar.ProgressBar(max_value=len(byte_array)) as bar:
            for byte in byte_array:
                self.encoded_string += bin(ord(byte))[2:].zfill(8)
                i += 1
                bar.update(i)
        f.close()
        
        
    def getStringOutputBytes(self):
        str_to_print = ''
        i = 0
        while i < len(self.output):
            str_to_print += unichr(int(self.output[i: i+8], 2))
            i += 8
        return str_to_print
            
    def calculateCompressionRatio(self):
        return float(os.path.getsize(self.file_path))/(len(self.output) / 8.0)

In [115]:
model = LZW()
model.readFile('DataSet/DataSet_11.tsv')

In [116]:
model.encode()

100% (1097857 of 1097857) |##############| Elapsed Time: 0:00:43 Time:  0:00:43


In [117]:
print model.calculateCompressionRatio()

3.90568759481


In [118]:
model.printEncodedFile()

In [119]:
model.readEncodedFile()

100% (504065 of 504065) |################| Elapsed Time: 0:01:23 Time:  0:01:23


In [120]:
model.output = model.encoded_string

In [121]:
model.decode()

100% (4032520 of 4032520) |##############| Elapsed Time: 0:00:45 Time:  0:00:45


In [122]:
print model.file_str == model.decoded

True
